In [1]:
#lets write a Simple script 
#to get the 20 words and their frequency percentage 
#with highest frequency in an English Wikipedia article. 
#applications are recommender systems, chatbots and NLP, sentiment analysis,
#data visualization,
#market research

#import dependencies
#Beautiful Soup is a Python library 
#for pulling data out of HTML and XML files.从网站取出来的数据是HTML，想把数据转化
from bs4 import BeautifulSoup

In [2]:
#Requests is one of the most downloaded 
#Python packages of all time, 
#pulling in over 7,000,000 downloads every month.
#HTTP library for pulling pushing and authenticating 抓取数据的
import requests

In [3]:
#lets you do Regular expression operations
#special text string for describing a search pattern.
#find and replace：regular expressions对文档的文字进行一些处理
import re

In [4]:
#The operator module exports a 
#set of efficient functions 
#corresponding to the intrinsic operators of Python.
#comparison, addition, greater than less then 加减之类 更易理解的处理
import operator

In [5]:
#parses解析 json, formats it，提取的format是json
import json

In [7]:
#The module provides just one function, 
#tabulate, which takes a list of lists or another 
#tabular data type as the first argument, 
#and outputs a nicely formatted plain-text table:更好的展示城table
from tabulate import tabulate

In [8]:
#system calls, dealw with user arguments： 这里的user input：What's the wikipidia artical we want to get
import sys

In [10]:
#list of common stop words various languages like the #words are not matter
from stop_words import get_stop_words

In [16]:
#get the words
def getWordList(url):
    word_list = []
    #raw data
    source_code = requests.get(url)
    #convert to text
    plain_text = source_code.text
    #lxml format
    soup = BeautifulSoup(plain_text,'lxml')

    #find the words in paragraph tag
    for text in soup.findAll('p'):
        if text.text is None:
            continue
        #content
        content = text.text
        #lowercase and split into an array
        words = content.lower().split()

        #for each word
        for word in words:
            #remove non-chars
            cleaned_word = clean_word(word)
            #if there is still something there
            if len(cleaned_word) > 0:
                #add it to our word list
                word_list.append(cleaned_word)

    return word_list


#clean word with regex
def clean_word(word):
    cleaned_word = re.sub('[^A-Za-z]+', '', word)
    return cleaned_word


def createFrquencyTable(word_list):
    #word count
    word_count = {}
    for word in word_list:
        #index is the word
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

    return word_count

#remove stop words
def remove_stop_words(frequency_list):
    stop_words = get_stop_words('en')

    temp_list = []
    for key,value in frequency_list:
        if key not in stop_words:
            temp_list.append([key, value])

    return temp_list

In [11]:
#access wiki API. json format. query it for data. search tyep. shows list of possibilities 
#第一个URL，Joson format---action=search(我们是从网页上queer data from web)---list type=search:我们type的字是去寻找一系列list---
#srsearch--我们自己加入的article的name， ex 加入Batman，会出现相关的文章的title的list
wikipedia_api_link = "https://en.wikipedia.org/w/api.php?format=json&action=query&list=search&srsearch="
wikipedia_link = "https://en.wikipedia.org/wiki/"
#第二个URL只是网站的网址

In [12]:
#if the search word is too small, throw an error
if(len(sys.argv) < 2):
    print("Enter valid string")
    exit()

In [13]:
#get the search word  这里不太懂过
string_query = sys.argv[1]

#to remove stop words or not  True代表表示去掉stop words，如果长于2个
if(len(sys.argv) > 2):
    search_mode = True
else:
    search_mode = False

In [14]:
#create our URL 建立自己的网址
url = wikipedia_api_link + string_query


In [19]:
#try-except block. simple way to deal with exceptions 
#great for HTTP requests
try:
    #use requests to retrieve raw data from wiki API URL we
    #just constructed
    response = requests.get(url)

    #format that data as a JSON dictionary 转变成我们想要的format
    data = json.loads(response.content.decode("utf-8"))

    #page title, first option
    #show this in web browser
    wikipedia_page_tag = data['query']['search'][0]['title']

    #get actual wiki page based on retrieved title
    url = wikipedia_link + wikipedia_page_tag
    
    #get list of words from that page
    page_word_list = getWordList(url)
    #create table of word counts, dictionary
    page_word_count = createFrquencyTable(page_word_list)
    #sort the table by the frequency count
    sorted_word_frequency_list = sorted(page_word_count.items(), key=operator.itemgetter(1), reverse=True)
    #remove stop words if the user specified，之前的stopword有search_mode= True
    if(search_mode):
        sorted_word_frequency_list = remove_stop_words(sorted_word_frequency_list)

    #sum the total words to calculate frequencies   
    total_words_sum = 0
    for key,value in sorted_word_frequency_list:
        total_words_sum = total_words_sum + value

    #just get the top 20 words
    if len(sorted_word_frequency_list) > 20:
        sorted_word_frequency_list = sorted_word_frequency_list[:20]

    #create our final list which contains words, frequency (word count), percentage
    final_list = []
    for key,value in sorted_word_frequency_list:
        percentage_value = float(value * 100) / total_words_sum
        final_list.append([key, value, round(percentage_value, 4)])

    #headers before the table
    print_headers = ['Word', 'Frequency', 'Frequency Percentage']

    #print the table with tabulate
    print(tabulate(final_list, headers=print_headers, tablefmt='orgtbl'))
    
    #throw an exception in case it breaks
except requests.exceptions.Timeout:
    print("The server didn't respond. Please, try again later.")


| Word           |   Frequency |   Frequency Percentage |
|----------------+-------------+------------------------|
| municipality   |          12 |                 5.4054 |
| may            |           6 |                 2.7027 |
| state          |           5 |                 2.2523 |
| communes       |           4 |                 1.8018 |
| latin          |           4 |                 1.8018 |
| municipalities |           4 |                 1.8018 |
| tax            |           4 |                 1.8018 |
| jurisdiction   |           3 |                 1.3514 |
| administrative |           3 |                 1.3514 |
| communities    |           3 |                 1.3514 |
| term           |           3 |                 1.3514 |
| also           |           3 |                 1.3514 |
| derived        |           3 |                 1.3514 |
| languages      |           2 |                 0.9009 |
| comun          |           2 |                 0.9009 |
| frenchspeaki